# Twitter Text Sentiment Analyser 

In [1]:
import tweepy
import csv

CONSUMER_KEY= '############'
CONSUMER_SECRET = '########'
ACCESS_KEY = '########'
ACCESS_SECRET = '######'
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_KEY, ACCESS_SECRET)

api = tweepy.API(auth)

def read_keywords_label(csv_in):
    keyword_label_list = []
    with open(csv_in,'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for row in reader:
            keyword_label_list.append( (row[0], row[1]))
    return keyword_label_list


def get_tweets(keyword, tweet_num):
    tweets_out = []
    tweets = tweepy.Cursor(api.search, q=keyword,result_type="recent",include_entities=True, lang="en").items(tweet_num)
    for tweet in tweets:
        tweets_out.append(tweet.text)
    return tweets_out


def get_labeled_tweets(keyword, label, tweets_per_keyword):
    labeled_tweets = []
    unlabeled_tweets = get_tweets(keyword, tweets_per_keyword)
    for tweet in unlabeled_tweets:
        labeled_tweets.append( (label, tweet))
    return labeled_tweets


def build_test_data(keywords_label_file_in, tweets_per_keyword):
    keywords_label = read_keywords_label(keywords_label_file_in)
    test_data = []
    for pair in keywords_label:
        labeled_tweets = get_labeled_tweets(pair[0], pair[1], tweets_per_keyword)
        test_data.extend(labeled_tweets)
    return test_data


import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline



def build_sentiment_model(keywords_label_file_in, tweets_per_keyword):
    test_data = build_test_data(keywords_label_file_in, tweets_per_keyword)
    train_target = [x[0] for x in test_data]
    train_data = [x[1] for x in test_data]
    model = make_pipeline(TfidfVectorizer(), MultinomialNB())
    model.fit(train_data, train_target)
    return model

def predict_category(model_in, text_in):
    labels = model_in.predict([text_in])
    return labels
    

In [6]:
get_labeled_tweets('I love learning', 'Bullish', 10)

[('Bullish', '@Drewscilla_ I love learning #yay'),
 ('Bullish',
  'RT @HeatherHadford: I love that our Superintendent is IN the teaching and learning WITH US. @GBProbeSchool @LethSD51 #vulnerableleadershipm…'),
 ('Bullish',
  'but translations do help in improving and learning new vocabulary so I would love to do more but please bear with m… https://t.co/TKwpastr1Y'),
 ('Bullish',
  'RT @NjSatz: #Happybirthdaymycinemaguru \nI learn many things from you.Still learning #godbless #stayhealthy \n@SasikumarDir love you sir 😍😍😍🙏…'),
 ('Bullish',
  'I love that our Superintendent is IN the teaching and learning WITH US. @GBProbeSchool @LethSD51… https://t.co/IEBl0mqklx'),
 ('Bullish',
  '@ShaneFilan I help out at a centre for people with Learning Difficulties. A guy called Jamie that goes there would love your autograph.'),
 ('Bullish',
  'I love learning the assignment material after the assignment is due thx'),
 ('Bullish',
  "RT @Oliveemodi: Most importantly, I learned and I'm still learni

In [3]:
build_test_data('sentiments.csv',10)

[('bullish',
  'I love how my iPhone or 2 hours of battery life coincidentally right when the new iPhones release'),
 ('bullish',
  'My new iPhone is pink and I love it mostly because I feel like a 21st century princess when I get it out on the tube 😇💅🏻💕 #bejel'),
 ('bullish',
  "Sorry @Apple I love you but I'm not gonna buy a 1150€ emoji machine when I can buy a 500€ iPhone 7 (Btw my iPhone 6 works like new so nope)"),
 ('bullish',
  "my gradmas just text me to say she's 'treated herself' and got the new iphone, bloody love that woman"),
 ('bullish',
  'My preggy gajah just got her new iphone. Love you gajah @OhEfjeeeek'),
 ('bullish', 'love the new update on my Iphone...'),
 ('bullish',
  'RT @WildflowerCases: Thinking about buying Turquoise iPhone 6/6s Case? Read our latest review of the p https://t.co/iboZmTCvSq via @yotpo'),
 ('bullish',
  'I love how the new iPhone update won’t show my Spotify in my lock screen even though I have that setting turned on.🙄'),
 ('bullish', 'Love lov

In [4]:
my_model= build_sentiment_model('sentiments.csv',10)

In [8]:
predict_category(my_model, 'Iphone 8')

array(['bullish'],
      dtype='<U7')